In [15]:
import os 
import cv2

In [16]:
path="Corona mask"
mylist=os.listdir("Corona mask")

In [17]:
features=[]
target=[]

In [18]:
for folder in mylist:
    mypiclist=os.listdir(path + "/" + folder)
    for i in mypiclist:
        image=cv2.imread(path + "/" + folder + "/"+ i)
        features.append(image)
        target.append(folder)
    print(folder)

not_mask
with_mask


In [19]:
import numpy as np

In [25]:
features

array([None,
       array([[[ 18,   3,   7],
        [ 18,   3,   7],
        [ 18,   3,   7],
        ...,
        [ 28,  12,  19],
        [ 28,  12,  19],
        [ 29,  13,  20]],

       [[ 22,   7,  11],
        [ 22,   7,  11],
        [ 22,   7,  11],
        ...,
        [ 41,  25,  32],
        [ 41,  25,  32],
        [ 42,  26,  33]],

       [[ 26,   9,  13],
        [ 26,   9,  13],
        [ 26,   9,  13],
        ...,
        [ 30,  15,  19],
        [ 31,  16,  20],
        [ 32,  17,  21]],

       ...,

       [[ 77,   5, 172],
        [ 73,   5, 170],
        [ 65,   5, 167],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 73,   5, 166],
        [ 67,   3, 162],
        [ 62,   5, 158],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 71,   6, 162],
        [ 65,   3, 157],
        [ 59,   4, 155],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
    

In [23]:
features=np.array(features,dtype='object')
target=np.array(target,dtype='object')

In [24]:
features.shape

(1377,)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_features,test_features,train_traget,test_target=train_test_split(features,target,test_size=0.2)

In [ ]:
def preprocessing(img):
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=img/255
    return img

In [ ]:
train_features=np.array(list(map(preprocessing,train_features)),dtype=object)
test_features=np.array(list(map(preprocessing,test_features)),dtype=object)

In [ ]:
test_features.shape

In [ ]:
train_features=train_features.reshape(1100, 32, 32,1)

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
import keras
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen=ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.2,shear_range=0.1,rotation_range=10)

In [ ]:
batches=datagen.flow(train_features,train_traget,batch_size=20)
X_batch,Y_batch=next(batches)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout

In [ ]:
model=Sequential()

In [ ]:
model.add(Conv2D(60,(2,2),input_shape=(32,32,1),activation="relu"))
model.add(Conv2D(60,(2,2),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(30,(2,2),activation="relu"))
model.add(Conv2D(30,(2,2),activation="relu"))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(300,activation="relu"))
model.add(Dense(43,activation="softmax"))

In [ ]:
from keras.optimizers import Adam

In [ ]:
model.compile(Adam(lr=0.001),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit_generator(datagen.flow(train_features,train_traget,batch_size=20),epochs=5)

In [ ]:
cap=cv2.VideoCapture(0)

In [ ]:
while True:
    message,imgOriginal=cap.read()
    img=np.array(imgOriginal)
    img=cv2.resize(img, (32,32))
    img=preprocessing(img)
    img=img.reshape(1 ,32 ,32 ,1)
    cv2.putText(imgOriginal,"Class",(20,35),cv2.FONT_HERSHEY_TRIPLEX,1,(23,120,255),2)
    cv2.putText(imgOriginal,"Probability",(20,75),cv2.FONT_HERSHEY_TRIPLEX,1,(23,120,255),2)
    probabilities=model.predict(img)
    classIndex=model.predict_classes(img)
    maxprob=np.amax(probabilities)
    if maxprob>0.75:
        cv2.putText(imgOriginal,folder,(120,35),cv2.FONT_HERSHEY_TRIPLEX,1,(23,120,255),2)
        cv2.putText(imgOriginal,str(maxprob) + "%",(120,75),cv2.FONT_HERSHEY_TRIPLEX,1,(23,120,255),2)
    cv2.imshow("Result",imgOriginal)
    if cv2.waitKey(1) & 0xFF==ord("q"):
        break